In [ ]:
import time
import ray
import random

In [1]:
ray.init(num_cpus = 4) # Specify this system has 4 CPUs.

@ray.remote
def do_some_work(x):
    time.sleep(1) # Replace this with work you need to do.
    return x

start = time.time()
results = [do_some_work.remote(x) for x in range(4)]
print("duration =", time.time() - start)
print("results = ", results)

start = time.time()
results = [ray.get(do_some_work.remote(x)) for x in range(4)]
print("duration =", time.time() - start)
print("results = ", results)

start = time.time()
results = ray.get([do_some_work.remote(x) for x in range(4)])
print("duration =", time.time() - start)
print("results = ", results)

2022-04-05 10:54:06,090	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8266


duration = 0.004987239837646484
results =  [ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000), ObjectRef(16310a0f0a45af5cffffffffffffffffffffffff0100000001000000), ObjectRef(c2668a65bda616c1ffffffffffffffffffffffff0100000001000000), ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000)]
duration = 5.041032075881958
results =  [0, 1, 2, 3]
duration = 1.0043976306915283
results =  [0, 1, 2, 3]


In [4]:
@ray.remote
def do_some_work(x):
    time.sleep(random.uniform(0, 4)) # Replace this with work you need to do.
    return x

def process_results(results):
    sum = 0
    for x in results:
        time.sleep(1) # Replace this with some processing code.
        sum += x
    return sum

start = time.time()
data_list = ray.get([do_some_work.remote(x) for x in range(4)])
sum = process_results(data_list)
print("duration =", time.time() - start, "\nresult = ", sum)

duration = 7.958974838256836 
result =  6


In [5]:
@ray.remote
def do_some_work(x):
    time.sleep(random.uniform(0, 4)) # Replace this with work you need to do.
    return x

def process_incremental(sum, result):
    time.sleep(1) # Replace this with some processing code.
    return sum + result

start = time.time()
result_ids = [do_some_work.remote(x) for x in range(4)]
sum = 0
while len(result_ids):
    done_id, result_ids = ray.wait(result_ids)
    sum = process_incremental(sum, ray.get(done_id[0]))
print("duration =", time.time() - start, "\nresult = ", sum)

duration = 5.447748899459839 
result =  6


In [7]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
digits = load_digits()
param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300, verbose=10)

import joblib
from ray.util.joblib import register_ray
register_ray()
with joblib.parallel_backend('ray'):
    search.fit(digits.data, digits.target)

2022-04-05 11:06:19,382	WARNING pool.py:494 -- The 'context' argument is not supported using ray. Please refer to the documentation for how to control ray initialization.


Fitting 5 folds for each of 300 candidates, totalling 1500 fits
 pid=41508) [CV 3/5; 1/300] START C=10.82636733874054, class_weight=None, gamma=174332.88221999872, tol=0.004520353656360241
 pid=28384) [CV 3/5; 2/300] START C=0.03562247890262444, class_weight=balanced, gamma=13738.23795883261, tol=0.007278953843983146
 pid=43756) [CV 5/5; 1/300] START C=10.82636733874054, class_weight=None, gamma=174332.88221999872, tol=0.004520353656360241
 pid=20852) [CV 1/5; 1/300] START C=10.82636733874054, class_weight=None, gamma=174332.88221999872, tol=0.004520353656360241
 pid=41508) [CV 3/5; 1/300] END C=10.82636733874054, class_weight=None, gamma=174332.88221999872, tol=0.004520353656360241;, score=0.103 total time=   0.3s
 pid=41508) [CV 4/5; 1/300] START C=10.82636733874054, class_weight=None, gamma=174332.88221999872, tol=0.004520353656360241
 pid=28384) [CV 3/5; 2/300] END C=0.03562247890262444, class_weight=balanced, gamma=13738.23795883261, tol=0.007278953843983146;, score=0.100 total ti

 pid=41508) [CV 2/5; 8/300] END C=3290.344562312671, class_weight=None, gamma=7.278953843983146e-05, tol=0.03856620421163472;, score=0.928 total time=   0.0s
 pid=41508) [CV 3/5; 8/300] START C=3290.344562312671, class_weight=None, gamma=7.278953843983146e-05, tol=0.03856620421163472
 pid=41508) [CV 3/5; 8/300] END C=3290.344562312671, class_weight=None, gamma=7.278953843983146e-05, tol=0.03856620421163472;, score=0.967 total time=   0.0s
 pid=43756) [CV 3/5; 6/300] END C=0.0020433597178569417, class_weight=balanced, gamma=6.723357536499335, tol=0.007278953843983146;, score=0.100 total time=   0.2s
 pid=43756) [CV 5/5; 6/300] START C=0.0020433597178569417, class_weight=balanced, gamma=6.723357536499335, tol=0.007278953843983146
 pid=28384) [CV 4/5; 6/300] END C=0.0020433597178569417, class_weight=balanced, gamma=6.723357536499335, tol=0.007278953843983146;, score=0.123 total time=   0.2s
 pid=28384) [CV 4/5; 7/300] START C=8531.678524172814, class_weight=balanced, gamma=621016.94189156

 pid=43756) [CV 1/5; 12/300] END C=188.73918221350996, class_weight=balanced, gamma=3.562247890262444e-08, tol=0.00020433597178569417;, score=0.883 total time=   0.1s
 pid=43756) [CV 2/5; 12/300] START C=188.73918221350996, class_weight=balanced, gamma=3.562247890262444e-08, tol=0.00020433597178569417
 pid=41508) [CV 1/5; 14/300] END C=1000000.0, class_weight=None, gamma=7880462.815669904, tol=0.014873521072935119;, score=0.100 total time=   0.2s
 pid=41508) [CV 4/5; 14/300] START C=1000000.0, class_weight=None, gamma=7880462.815669904, tol=0.014873521072935119
 pid=43756) [CV 2/5; 12/300] END C=188.73918221350996, class_weight=balanced, gamma=3.562247890262444e-08, tol=0.00020433597178569417;, score=0.828 total time=   0.1s
 pid=43756) [CV 3/5; 12/300] START C=188.73918221350996, class_weight=balanced, gamma=3.562247890262444e-08, tol=0.00020433597178569417
 pid=20852) [CV 5/5; 13/300] END C=188.73918221350996, class_weight=None, gamma=23.95026619987481, tol=0.014873521072935119;, sco

 pid=28384) [CV 5/5; 18/300] END C=57361.52510448681, class_weight=balanced, gamma=621016.9418915603, tol=0.003562247890262444;, score=0.100 total time=   0.2s
 pid=28384) [CV 2/5; 19/300] START C=3290.344562312671, class_weight=None, gamma=0.0002592943797404667, tol=0.1
 pid=20852) [CV 3/5; 17/300] END C=4.520353656360241e-05, class_weight=balanced, gamma=303.9195382313195, tol=0.0013738237958832637;, score=0.100 total time=   0.3s
 pid=20852) [CV 2/5; 18/300] START C=57361.52510448681, class_weight=balanced, gamma=621016.9418915603, tol=0.003562247890262444
 pid=41508) [CV 1/5; 18/300] END C=57361.52510448681, class_weight=balanced, gamma=621016.9418915603, tol=0.003562247890262444;, score=0.100 total time=   0.3s
 pid=41508) [CV 5/5; 20/300] START C=489.3900918477499, class_weight=None, gamma=1.8873918221350996, tol=0.009236708571873866
 pid=28384) [CV 2/5; 19/300] END C=3290.344562312671, class_weight=None, gamma=0.0002592943797404667, tol=0.1;, score=0.947 total time=   0.0s
 pid=

...

 pid=28384) [CV 5/5; 275/300] END C=0.00011721022975334806, class_weight=None, gamma=1082.6367338740563, tol=0.0001268961003167922;, score=0.100 total time=   0.2s
 pid=28384) [CV 2/5; 276/300] START C=57361.52510448681, class_weight=balanced, gamma=0.5298316906283702, tol=0.0008531678524172806
 pid=43756) [CV 1/5; 275/300] END C=0.00011721022975334806, class_weight=None, gamma=1082.6367338740563, tol=0.0001268961003167922;, score=0.100 total time=   0.2s
 pid=43756) [CV 4/5; 276/300] START C=57361.52510448681, class_weight=balanced, gamma=0.5298316906283702, tol=0.0008531678524172806
 pid=41508) [CV 1/5; 276/300] END C=57361.52510448681, class_weight=balanced, gamma=0.5298316906283702, tol=0.0008531678524172806;, score=0.100 total time=   0.2s
 pid=20852) [CV 3/5; 274/300] END C=1268.9610031679233, class_weight=balanced, gamma=303.9195382313195, tol=0.00221221629107045;, score=0.103 total time=   0.2s
 pid=20852) [CV 4/5; 274/300] START C=1268.9610031679233, class_weight=balanced, gam

 pid=41508) [CV 3/5; 281/300] END C=0.2395026619987486, class_weight=balanced, gamma=0.041753189365604, tol=0.06210169418915616;, score=0.106 total time=   0.1s
 pid=41508) [CV 4/5; 281/300] START C=0.2395026619987486, class_weight=balanced, gamma=0.041753189365604, tol=0.06210169418915616
 pid=20852) [CV 2/5; 281/300] END C=0.2395026619987486, class_weight=balanced, gamma=0.041753189365604, tol=0.06210169418915616;, score=0.100 total time=   0.1s
 pid=20852) [CV 4/5; 282/300] START C=188.73918221350996, class_weight=balanced, gamma=100000000.0, tol=0.07880462815669913
 pid=28384) [CV 2/5; 282/300] END C=188.73918221350996, class_weight=balanced, gamma=100000000.0, tol=0.07880462815669913;, score=0.100 total time=   0.3s
 pid=28384) [CV 5/5; 282/300] START C=188.73918221350996, class_weight=balanced, gamma=100000000.0, tol=0.07880462815669913
 pid=43756) [CV 3/5; 280/300] END C=0.0003039195382313198, class_weight=None, gamma=48939.00918477499, tol=0.04893900918477494;, score=0.103 tota

 pid=41508) [CV 5/5; 286/300] END C=1268.9610031679233, class_weight=balanced, gamma=3856.620421163472, tol=0.04893900918477494;, score=0.100 total time=   0.2s
 pid=41508) [CV 4/5; 287/300] START C=1.7433288221999873e-05, class_weight=None, gamma=1.6102620275609392e-06, tol=0.014873521072935119
 pid=28384) [CV 5/5; 288/300] START C=1e-06, class_weight=None, gamma=2212216.29107045, tol=0.0028072162039411755
 pid=43756) [CV 4/5; 286/300] END C=1268.9610031679233, class_weight=balanced, gamma=3856.620421163472, tol=0.04893900918477494;, score=0.100 total time=   0.2s
 pid=43756) [CV 2/5; 287/300] START C=1.7433288221999873e-05, class_weight=None, gamma=1.6102620275609392e-06, tol=0.014873521072935119
 pid=20852) [CV 4/5; 288/300] START C=1e-06, class_weight=None, gamma=2212216.29107045, tol=0.0028072162039411755
 pid=43756) [CV 2/5; 287/300] END C=1.7433288221999873e-05, class_weight=None, gamma=1.6102620275609392e-06, tol=0.014873521072935119;, score=0.100 total time=   0.1s
 pid=43756)

 pid=41508) [CV 5/5; 292/300] END C=10.82636733874054, class_weight=None, gamma=13738.23795883261, tol=0.0017433288221999873;, score=0.100 total time=   0.2s
 pid=41508) [CV 3/5; 293/300] START C=0.03562247890262444, class_weight=balanced, gamma=0.5298316906283702, tol=0.0001268961003167922
 pid=28384) [CV 4/5; 293/300] END C=0.03562247890262444, class_weight=balanced, gamma=0.5298316906283702, tol=0.0001268961003167922;, score=0.097 total time=   0.2s
 pid=28384) [CV 5/5; 293/300] START C=0.03562247890262444, class_weight=balanced, gamma=0.5298316906283702, tol=0.0001268961003167922
 pid=20852) [CV 3/5; 294/300] END C=489.3900918477499, class_weight=None, gamma=2212216.29107045, tol=0.04893900918477494;, score=0.103 total time=   0.2s
 pid=20852) [CV 5/5; 294/300] START C=489.3900918477499, class_weight=None, gamma=2212216.29107045, tol=0.04893900918477494
 pid=20852) [CV 5/5; 294/300] END C=489.3900918477499, class_weight=None, gamma=2212216.29107045, tol=0.04893900918477494;, score=

 pid=28384) [CV 2/5; 299/300] END C=0.09236708571873865, class_weight=balanced, gamma=7.278953843983146e-05, tol=0.009236708571873866;, score=0.831 total time=   0.1s
 pid=28384) [CV 4/5; 299/300] START C=0.09236708571873865, class_weight=balanced, gamma=7.278953843983146e-05, tol=0.009236708571873866
 pid=43756) [CV 3/5; 298/300] END C=10.82636733874054, class_weight=None, gamma=1e-08, tol=0.003562247890262444;, score=0.203 total time=   0.1s
 pid=43756) [CV 1/5; 299/300] START C=0.09236708571873865, class_weight=balanced, gamma=7.278953843983146e-05, tol=0.009236708571873866
 pid=28384) [CV 4/5; 299/300] END C=0.09236708571873865, class_weight=balanced, gamma=7.278953843983146e-05, tol=0.009236708571873866;, score=0.936 total time=   0.1s
 pid=28384) [CV 1/5; 300/300] START C=4.520353656360241e-05, class_weight=None, gamma=3.562247890262444e-08, tol=0.001082636733874054
 pid=43756) [CV 1/5; 299/300] END C=0.09236708571873865, class_weight=balanced, gamma=7.278953843983146e-05, tol=0.

In [ ]:
# https://github.com/ray-project/xgboost_ray

In [10]:
from xgboost import DMatrix, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = DMatrix(train_x, train_y)

evals_result = {}
bst = train(
   {
       "objective": "binary:logistic",
       "eval_metric": ["logloss", "error"],
   },
   train_set,
   evals_result=evals_result,
   evals=[(train_set, "train")],
   verbose_eval=False)

bst.save_model("model.xgb")

In [15]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
   {
       "objective": "binary:logistic",
       "eval_metric": ["logloss", "error"],
   },
   train_set,
   evals_result=evals_result,
   evals=[(train_set, "train")],
   verbose_eval=False,
   ray_params=RayParams(num_actors=1, cpus_per_actor=4))

bst.save_model("model.xgb")

(pid=46420) C:\Users\user\miniconda3\envs\date-eng\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pid=46420)   from pandas import MultiIndex, Int64Index


(scheduler +20m49s) Error: No available node types can fulfill resource request {'CPU': 6.0}. Add suitable node types to this cluster to resolve this issue.


KeyboardInterrupt: 

In [22]:
ray.shutdown()

In [19]:
ray.init()

2022-04-05 11:15:46,615	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': 'tcp://127.0.0.1:52202',
 'raylet_socket_name': 'tcp://127.0.0.1:50436',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\user\\AppData\\Local\\Temp\\ray\\session_2022-04-05_11-15-43_955036_27588',
 'metrics_export_port': 52687,
 'gcs_address': '127.0.0.1:61618',
 'address': '127.0.0.1:61618',
 'node_id': 'd8af25593dfd6439a267e63242f3179d691cbd3a4b8882978b32c243'}

In [20]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
   {
       "objective": "binary:logistic",
       "eval_metric": ["logloss", "error"],
   },
   train_set,
   evals_result=evals_result,
   evals=[(train_set, "train")],
   verbose_eval=False,
   ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")

2022-04-05 11:15:54,369	INFO main.py:985 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
 pid=22332) C:\Users\user\miniconda3\envs\date-eng\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
 pid=22332)   from pandas import MultiIndex, Int64Index
(pid=24764) C:\Users\user\miniconda3\envs\date-eng\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pid=24764)   from pandas import MultiIndex, Int64Index
(pid=25168) C:\Users\user\miniconda3\envs\date-eng\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pid=25168)   f

ValueError: cannot find context for 'fork'

In [23]:
"""This example demonstrates basic Ray Tune random search and grid search."""
import time

import ray
from ray import tune


def evaluation_fn(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100)**(-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    parser.add_argument(
        "--server-address",
        type=str,
        default=None,
        required=False,
        help="The address of server to connect to if using "
        "Ray Client.")
    args, _ = parser.parse_known_args()
    if args.server_address is not None:
        ray.init(f"ray://{args.server_address}")
    else:
        ray.init(configure_logging=False)

    # This will do a grid search over the `activation` parameter. This means
    # that each of the two values (`relu` and `tanh`) will be sampled once
    # for each sample (`num_samples`). We end up with 2 * 50 = 100 samples.
    # The `width` and `height` parameters are sampled randomly.
    # `steps` is a constant parameter.

    analysis = tune.run(
        easy_objective,
        metric="mean_loss",
        mode="min",
        num_samples=5 if args.smoke_test else 50,
        config={
            "steps": 5 if args.smoke_test else 100,
            "width": tune.uniform(0, 20),
            "height": tune.uniform(-100, 100),
            "activation": tune.grid_search(["relu", "tanh"])
        })

    print("Best hyperparameters found were: ", analysis.best_config)

2022-04-05 11:18:00,935	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265
2022-04-05 11:18:06,086	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-04-05 11:18:06,162	WARNING tune.py:583 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
2022-04-05 11:18:06,355	ERROR syncer.py:111 -- Log sync requires rsync to be installed.


Trial name,status,loc,activation,height,width
easy_objective_a0f8c_00000,RUNNING,127.0.0.1:3676,relu,25.269,1.46278
easy_objective_a0f8c_00001,PENDING,,tanh,-63.1354,9.28939
easy_objective_a0f8c_00002,PENDING,,relu,-41.3671,8.23914
easy_objective_a0f8c_00003,PENDING,,tanh,-26.2301,13.2109
easy_objective_a0f8c_00004,PENDING,,relu,20.0797,7.24909
easy_objective_a0f8c_00005,PENDING,,tanh,83.3035,12.6244
easy_objective_a0f8c_00006,PENDING,,relu,25.1189,2.17651
easy_objective_a0f8c_00007,PENDING,,tanh,71.1899,12.9923
easy_objective_a0f8c_00008,PENDING,,relu,-1.50894,7.59309
easy_objective_a0f8c_00009,PENDING,,tanh,90.5436,1.71134


Result for easy_objective_a0f8c_00000:
  date: 2022-04-05_11-18-07
  done: false
  experiment_id: 2192091744ac41208425d960ffd7c3b7
  hostname: DESKTOP-1QVV4ER
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 12.526904265145504
  neg_mean_loss: -12.526904265145504
  node_ip: 127.0.0.1
  pid: 3676
  time_since_restore: 0.10171842575073242
  time_this_iter_s: 0.10171842575073242
  time_total_s: 0.10171842575073242
  timestamp: 1649125087
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a0f8c_00000
  
Result for easy_objective_a0f8c_00001:
  date: 2022-04-05_11-18-08
  done: false
  experiment_id: 19092e6bc447480d88637da80223ce75
  hostname: DESKTOP-1QVV4ER
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 3.6864631447609426
  neg_mean_loss: -3.6864631447609426
  node_ip: 127.0.0.1
  pid: 6272
  time_since_restore: 0.10372161865234375
  time_this_iter_s: 0.10372161865234375
  time_total_s: 0.10372161865234375
  timestamp: 1649125088
  timesteps_since_rest

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_a0f8c_00000,RUNNING,127.0.0.1:3676,relu,25.269,1.46278,4.16097,36,4.26145,35,-4.16097
easy_objective_a0f8c_00001,RUNNING,127.0.0.1:6272,tanh,-63.1354,9.28939,-5.88425,25,2.79176,24,5.88425
easy_objective_a0f8c_00002,RUNNING,127.0.0.1:46220,relu,-41.3671,8.23914,-3.65534,25,2.78677,24,3.65534
easy_objective_a0f8c_00003,RUNNING,127.0.0.1:30868,tanh,-26.2301,13.2109,-2.31726,25,2.78078,24,2.31726
easy_objective_a0f8c_00004,RUNNING,127.0.0.1:6612,relu,20.0797,7.24909,2.55151,25,2.77879,24,-2.55151
easy_objective_a0f8c_00005,RUNNING,127.0.0.1:27528,tanh,83.3035,12.6244,8.66328,24,2.70499,23,-8.66328
easy_objective_a0f8c_00006,RUNNING,127.0.0.1:38484,relu,25.1189,2.17651,4.11867,25,2.77679,24,-4.11867
easy_objective_a0f8c_00007,RUNNING,127.0.0.1:52760,tanh,71.1899,12.9923,7.4428,24,2.701,23,-7.4428
easy_objective_a0f8c_00008,RUNNING,127.0.0.1:53360,relu,-1.50894,7.59309,0.390698,24,2.70598,23,-0.390698
easy_objective_a0f8c_00009,RUNNING,127.0.0.1:38764,tanh,90.5436,1.71134,11.0803,24,2.699,23,-11.0803


Result for easy_objective_a0f8c_00000:
  date: 2022-04-05_11-18-12
  done: false
  experiment_id: 2192091744ac41208425d960ffd7c3b7
  hostname: DESKTOP-1QVV4ER
  iterations: 44
  iterations_since_restore: 45
  mean_loss: 3.8716745911781936
  neg_mean_loss: -3.8716745911781936
  node_ip: 127.0.0.1
  pid: 3676
  time_since_restore: 5.28153920173645
  time_this_iter_s: 0.12111616134643555
  time_total_s: 5.28153920173645
  timestamp: 1649125092
  timesteps_since_restore: 0
  training_iteration: 45
  trial_id: a0f8c_00000
  
Result for easy_objective_a0f8c_00001:
  date: 2022-04-05_11-18-13
  done: false
  experiment_id: 19092e6bc447480d88637da80223ce75
  hostname: DESKTOP-1QVV4ER
  iterations: 45
  iterations_since_restore: 46
  mean_loss: -6.07990433469738
  neg_mean_loss: 6.07990433469738
  node_ip: 127.0.0.1
  pid: 6272
  time_since_restore: 5.115570783615112
  time_this_iter_s: 0.10505104064941406
  time_total_s: 5.115570783615112
  timestamp: 1649125093
  timesteps_since_restore: 0
  

...

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_a0f8c_00097,RUNNING,127.0.0.1:31460,tanh,40.391,6.58197,4.23259,78,8.22295,77,-4.23259
easy_objective_a0f8c_00098,RUNNING,127.0.0.1:21732,relu,35.8597,11.1016,3.70311,77,8.12524,76,-3.70311
easy_objective_a0f8c_00099,RUNNING,127.0.0.1:40908,tanh,19.8413,14.0997,2.07658,77,8.11327,76,-2.07658
easy_objective_a0f8c_00000,TERMINATED,127.0.0.1:3676,relu,25.269,1.46278,3.17284,100,11.3101,99,-3.17284
easy_objective_a0f8c_00001,TERMINATED,127.0.0.1:6272,tanh,-63.1354,9.28939,-6.20597,100,11.1337,99,6.20597
easy_objective_a0f8c_00002,TERMINATED,127.0.0.1:46220,relu,-41.3671,8.23914,-4.0156,100,11.0229,99,4.0156
easy_objective_a0f8c_00003,TERMINATED,127.0.0.1:30868,tanh,-26.2301,13.2109,-2.54713,100,11.2431,99,2.54713
easy_objective_a0f8c_00004,TERMINATED,127.0.0.1:6612,relu,20.0797,7.24909,2.1454,100,11.1962,99,-2.1454
easy_objective_a0f8c_00005,TERMINATED,127.0.0.1:27528,tanh,83.3035,12.6244,8.40973,100,11.3169,99,-8.40973
easy_objective_a0f8c_00006,TERMINATED,127.0.0.1:38484,relu,25.1189,2.17651,2.9554,100,11.1793,99,-2.9554


Result for easy_objective_a0f8c_00098:
  date: 2022-04-05_11-20-08
  done: false
  experiment_id: a5a4b0a5bbe84e7d9366ddf70f16c9ba
  hostname: DESKTOP-1QVV4ER
  iterations: 95
  iterations_since_restore: 96
  mean_loss: 3.679901642976698
  neg_mean_loss: -3.679901642976698
  node_ip: 127.0.0.1
  pid: 21732
  time_since_restore: 10.162523746490479
  time_this_iter_s: 0.10472512245178223
  time_total_s: 10.162523746490479
  timestamp: 1649125208
  timesteps_since_restore: 0
  training_iteration: 96
  trial_id: a0f8c_00098
  
Result for easy_objective_a0f8c_00097:
  date: 2022-04-05_11-20-08
  done: false
  experiment_id: d32d7a17cbfe4812bf153fd32867cd06
  hostname: DESKTOP-1QVV4ER
  iterations: 96
  iterations_since_restore: 97
  mean_loss: 4.194890857047817
  neg_mean_loss: -4.194890857047817
  node_ip: 127.0.0.1
  pid: 31460
  time_since_restore: 10.215378999710083
  time_this_iter_s: 0.10471749305725098
  time_total_s: 10.215378999710083
  timestamp: 1649125208
  timesteps_since_resto

Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
easy_objective_a0f8c_00000,TERMINATED,127.0.0.1:3676,relu,25.269,1.46278,3.17284,100,11.3101,99,-3.17284
easy_objective_a0f8c_00001,TERMINATED,127.0.0.1:6272,tanh,-63.1354,9.28939,-6.20597,100,11.1337,99,6.20597
easy_objective_a0f8c_00002,TERMINATED,127.0.0.1:46220,relu,-41.3671,8.23914,-4.0156,100,11.0229,99,4.0156
easy_objective_a0f8c_00003,TERMINATED,127.0.0.1:30868,tanh,-26.2301,13.2109,-2.54713,100,11.2431,99,2.54713
easy_objective_a0f8c_00004,TERMINATED,127.0.0.1:6612,relu,20.0797,7.24909,2.1454,100,11.1962,99,-2.1454
easy_objective_a0f8c_00005,TERMINATED,127.0.0.1:27528,tanh,83.3035,12.6244,8.40973,100,11.3169,99,-8.40973
easy_objective_a0f8c_00006,TERMINATED,127.0.0.1:38484,relu,25.1189,2.17651,2.9554,100,11.1793,99,-2.9554
easy_objective_a0f8c_00007,TERMINATED,127.0.0.1:52760,tanh,71.1899,12.9923,7.19614,100,11.1334,99,-7.19614
easy_objective_a0f8c_00008,TERMINATED,127.0.0.1:53360,relu,-1.50894,7.59309,-0.0196112,100,11.4232,99,0.0196112
easy_objective_a0f8c_00009,TERMINATED,127.0.0.1:38764,tanh,90.5436,1.71134,9.6117,100,11.3109,99,-9.6117


2022-04-05 11:20:09,512	INFO tune.py:639 -- Total run time: 123.43 seconds (123.19 seconds for the tuning loop).


Best hyperparameters found were:  {'steps': 100, 'width': 2.8952718518134457, 'height': -98.16160046442617, 'activation': 'relu'}


In [26]:
from lightgbm_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary",
        "metric": ["binary_logloss", "binary_error"],
    },
    train_set,
    evals_result=evals_result,
    valid_sets=[train_set],
    valid_names=["train"],
    verbose_eval=False,
    ray_params=RayParams(num_actors=1, cpus_per_actor=6))

bst.booster_.save_model("model.lgbm")
print("Final training error: {:.4f}".format(
    evals_result["train"]["binary_error"][-1]))

DEBUG:lightgbm_ray.main:Boost rounds left: 10
INFO:lightgbm_ray.main:[RayLightGBM] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(pid=16852) C:\Users\user\miniconda3\envs\date-eng\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pid=16852)   from pandas import MultiIndex, Int64Index
 pid=15564) C:\Users\user\miniconda3\envs\date-eng\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
 pid=15564)   from pandas import MultiIndex, Int64Index
INFO:lightgbm_ray.main:[RayLightGBM] Starting LightGBM training.
DEBUG:lightgbm_ray.main:Obtained unique addresses in 0 attempts.


 pid=15564) [LightGBM] [Warning] num_threads is set=6, n_jobs=-1 will be ignored. Current value: num_threads=6


 pid=34400) C:\Users\user\miniconda3\envs\date-eng\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
 pid=34400)   from pandas import MultiIndex, Int64Index
INFO:lightgbm_ray.main:[RayLightGBM] Finished LightGBM training on training data with total N=569 in 5.46 seconds (2.32 pure LightGBM training time).


Final training error: 0.0193


In [27]:
evals_result

{'train': OrderedDict([('binary_logloss',
               [0.5770848745933277,
                0.5141558520475956,
                0.45935842229376317,
                0.4160302696445565,
                0.37673950461280115,
                0.3429622505940099,
                0.31260648018952664,
                0.2848676613137762,
                0.2618415689365064,
                0.24118891621483915]),
              ('binary_error',
               [0.37258347978910367,
                0.37258347978910367,
                0.06678383128295255,
                0.05799648506151142,
                0.054481546572934976,
                0.029876977152899824,
                0.02460456942003515,
                0.02460456942003515,
                0.028119507908611598,
                0.019332161687170474])])}